In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ipc-booklet/IPC_186045.pdf


In [13]:
!pip install PyMuPDF pandas


In [14]:
import fitz  # PyMuPDF

pdf_path = "/kaggle/input/ipc-booklet/IPC_186045.pdf"  # <-- your IPC PDF filename
text = ""

with fitz.open(pdf_path) as doc:
    for page_number in range(14, len(doc)):  # Page numbers are 0-indexed (page 15 → index 14)
        page = doc[page_number]
        text += page.get_text("text") + "\n"
        
# 💾 Save extracted text
output_path = "/kaggle/working/ipc_book.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(text)

print("✅ Text extracted and saved at:", output_path)


✅ Text extracted and saved at: /kaggle/working/ipc_book.txt


In [15]:
with open("/kaggle/working/ipc_book.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [16]:
lines = text.splitlines()

data = []
chapter_number = ""
chapter_name = ""
section_number = ""
section_text = ""

In [17]:
# 🧩 Regex patterns
section_pattern = re.compile(r"^(\d+)\.\s*(.*)")
chapter_pattern = re.compile(r"^CHAPTER\s+([IVXLCDM]+)", re.IGNORECASE)

In [18]:
for i, line in enumerate(lines):
    line = line.strip()
    if not line:
        continue

    # Detect chapter headers (e.g., CHAPTER VIII)
    chapter_match = chapter_pattern.match(line)
    if chapter_match:
        chapter_number = chapter_match.group(1)
        # Next line is usually the chapter name
        if i + 1 < len(lines):
            next_line = lines[i + 1].strip()
            if next_line:
                # Fix spacing issues like "OFOFFENCES" → "OF OFFENCES"
                chapter_name = next_line.replace("OFOFFENCES", "OF OFFENCES")
        continue

    # Detect section starts (e.g., "141. Unlawful assembly")
    section_match = section_pattern.match(line)
    if section_match:
        # Save the previous section before starting a new one
        if section_number:
            data.append([chapter_number, chapter_name, section_number, section_text.strip()])
        section_number = section_match.group(1)
        section_text = section_match.group(2)
    else:
        section_text += " " + line

In [19]:
if section_number:
    data.append([chapter_number, chapter_name, section_number, section_text.strip()])

# 🧾 Create DataFrame
df = pd.DataFrame(data, columns=["chapter_number", "chapter_name", "section_number", "description"])

# 💾 Save the structured data
output_path = "/kaggle/working/ipc_sections_structured.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

print("✅ CSV created successfully at:", output_path)
df.head()

✅ CSV created successfully at: /kaggle/working/ipc_sections_structured.csv


,chapter_number,chapter_name,section_number,description
0,I,INTRODUCTION,1,Title and extent of operation of the Code.—Thi...
1,I,INTRODUCTION,2,Punishment of offences committed within India....
2,I,INTRODUCTION,3,"Punishment of offences committed beyond, but w..."
3,I,INTRODUCTION,1,The Indian Penal Code has been extended to Ber...
4,I,INTRODUCTION,2,The words “British India” have successively be...


In [24]:

# 📘 Load your structured CSV
df = pd.read_csv("/kaggle/working/ipc_sections_structured.csv")

# 🧩 Define common legal abbreviations and their expansions
abbreviation_map = {
    r"\bA\.O\.": "Adaptation Order",
    r"\bAct\b": "Act",
    r"\bs\.\s*": "section ",
    r"\bsubs\.": "substituted",
    r"\bSubs\.": "substituted",
    r"\bins\.": "inserted",
    r"\bIns\.": "inserted",
    r"\brep\.": "repealed",
    r"\bcl\.": "clause",
    r"\bcls\.": "clauses",
    r"\bSch\.": "Schedule",
    r"\bw\.e\.f\.": "with effect from",
    r"\bibid\.": "in the same place",
    r"\bs\.": "section ",
    r"\bw\.e\.f": "with effect from",
    r"\bNo\.": "Number",
    r"\bpp\.": "pages",
    r"\bvol\.": "volume",
}

# 🧹 Function to expand abbreviations in text
def expand_abbreviations(text):
    for pattern, full_form in abbreviation_map.items():
        text = re.sub(pattern, full_form, text)
    return text

# 🧠 Apply to description column (or wherever abbreviations appear)
df["description"] = df["description"].apply(expand_abbreviations)

# 💾 Save cleaned version
output_path = "/kaggle/working/ipc_sections_cleaned.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

print("✅ Cleaned CSV saved at:", output_path)
df.head()


✅ Cleaned CSV saved at: /kaggle/working/ipc_sections_cleaned.csv


,chapter_number,chapter_name,section_number,description
0,I,INTRODUCTION,1,Title and extent of operation of the Code.—Thi...
1,I,INTRODUCTION,2,Punishment of offences committed within India....
2,I,INTRODUCTION,3,"Punishment of offences committed beyond, but w..."
3,I,INTRODUCTION,1,The Indian Penal Code has been extended to Ber...
4,I,INTRODUCTION,2,The words “British India” have successively be...
